In [1]:
import networkx as nx
import os
from pathlib import Path
import pandas as pd
import numpy as np
import math
from collections import defaultdict

In [2]:
xferpath = Path(r"C:\Users\User\Documents\cfb project\data\transferportal\raw")
recpath = Path(r"C:\Users\User\Documents\cfb project\data\recruiting\original")
xferfiles = {}
recfiles = {}

for file_path in xferpath.glob('*.graphml'):
    name = file_path.stem
    G_x = nx.read_graphml(file_path)
    xferfiles[name] = G_x

for file_path in recpath.glob('*.graphml'):
    name = file_path.stem
    G_r = nx.read_graphml(file_path)
    recfiles[name] = G_r

print(f"\nSuccessfully loaded {len(xferfiles)} files from xfer and {len(recfiles)} files from rec.")



Successfully loaded 5 files from xfer and 27 files from rec.


In [5]:
import networkx as nx
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict

# Load your transfer portal graphs
xferpath = Path(r"C:\Users\User\Documents\cfb project\data\transferportal\raw")
xferfiles = {}

for file_path in xferpath.glob('*.graphml'):
    name = file_path.stem
    G_x = nx.read_graphml(file_path)
    xferfiles[name] = G_x

print(f"Loaded {len(xferfiles)} transfer portal files\n")

# ============================================
# STEP 1: ANALYZE CURRENT STATE
# ============================================

print("="*60)
print("ANALYZING CURRENT DATA STATE")
print("="*60 + "\n")

for name, graph in xferfiles.items():
    total_edges = len(graph.edges())
    total_players = 0
    multi_player_edges = 0
    
    for s, t, data in graph.edges(data=True):
        players_str = data.get('players', '')
        if players_str:
            player_list = [p.strip() for p in players_str.split('|')]
            total_players += len(player_list)
            if len(player_list) > 1:
                multi_player_edges += 1
    
    print(f"{name}:")
    print(f"  Edges: {total_edges}")
    print(f"  Players: {total_players}")
    print(f"  Multi-player edges: {multi_player_edges}")
    print()

# ============================================
# STEP 2: BUILD STAR-TO-RATING MAPPING
# ============================================

print("="*60)
print("BUILDING STAR-TO-RATING MAPPING FROM EXISTING DATA")
print("="*60 + "\n")

all_ratings_by_star = {1: [], 2: [], 3: [], 4: [], 5: []}

for name, graph in xferfiles.items():
    for s, t, data in graph.edges(data=True):
        ratings_str = data.get('ratings', '')
        stars_str = data.get('stars', '')
        
        if not ratings_str or not stars_str:
            continue
        
        # Split by pipe and clean
        ratings_list = [r.strip() for r in ratings_str.split('|')]
        stars_list = [st.strip() for st in stars_str.split('|')]
        
        # Match up ratings with stars
        for rating, star in zip(ratings_list, stars_list):
            try:
                rating_val = float(rating) if rating and rating != '' and rating != 'None' else None
                star_val = int(float(star)) if star and star != '' and star != 'None' else None
                
                if rating_val is not None and star_val is not None and 1 <= star_val <= 5:
                    all_ratings_by_star[star_val].append(rating_val)
            except (ValueError, TypeError):
                continue

# Calculate statistics for each star level
print("Rating statistics by star level:")
print("-" * 60)

star_rating_map = {}
for star in sorted(all_ratings_by_star.keys()):
    ratings = all_ratings_by_star[star]
    if ratings:
        mean_rating = np.mean(ratings)
        median_rating = np.median(ratings)
        std_rating = np.std(ratings)
        min_rating = np.min(ratings)
        max_rating = np.max(ratings)
        
        star_rating_map[star] = {
            'mean': mean_rating,
            'median': median_rating,
            'std': std_rating,
            'min': min_rating,
            'max': max_rating,
            'count': len(ratings)
        }
        
        print(f"{star} Stars:")
        print(f"  Count: {len(ratings)}")
        print(f"  Mean:   {mean_rating:.4f}")
        print(f"  Median: {median_rating:.4f}")
        print(f"  Std:    {std_rating:.4f}")
        print(f"  Range:  {min_rating:.4f} - {max_rating:.4f}")
        print()
    else:
        print(f"{star} Stars: No data available\n")
        star_rating_map[star] = {'mean': None, 'median': None}

# Define fallback function
def get_fallback_rating(star_value, method='median'):
    """Returns a fallback rating based on star value."""
    try:
        star = int(float(star_value))
        if star in star_rating_map and star_rating_map[star][method] is not None:
            return star_rating_map[star][method]
    except (ValueError, TypeError):
        pass
    return 0.0

# ============================================
# STEP 3: SPLIT EDGES BY PLAYER
# ============================================

print("="*60)
print("SPLITTING MULTI-PLAYER EDGES")
print("="*60 + "\n")

def are_same_person(name1, name2):
    """
    Returns True if two players share the same last name, 
    ignoring suffixes like Jr, Sr, III.
    """
    if not name1 or not name2:
        return False

    def get_cleaned_last_name(full_name):
        parts = full_name.strip().lower().split()
        
        if not parts:
            return ""

        suffixes = {'jr', 'jr.', 'sr', 'sr.', 'ii', 'iii', 'iv', 'v'}

        while len(parts) > 1 and parts[-1] in suffixes:
            parts.pop()
            
        return parts[-1]

    return get_cleaned_last_name(name1) == get_cleaned_last_name(name2)

cleaned_graphs = {}

for name, graph in xferfiles.items():
    print(f"Processing {name}...")
    
    # Create new MULTI-graph to support multiple edges between same nodes
    G_new = nx.MultiDiGraph()
    
    # Copy node attributes
    for node, attrs in graph.nodes(data=True):
        G_new.add_node(node, **attrs)
    
    edges_before = len(graph.edges())
    edges_after = 0
    players_processed = 0
    duplicates_merged = 0
    
    # Process each edge
    for source, target, data in graph.edges(data=True):
        # Extract all the pipe-separated attributes
        players_str = data.get('players', '')
        positions_str = data.get('positions', '')
        dates_str = data.get('dates', '')
        ratings_str = data.get('ratings', '')
        stars_str = data.get('stars', '')
        eligibility_str = data.get('eligibility', '')
        
        if not players_str:
            continue
        
        # Split all attributes by pipe
        players = [p.strip() for p in players_str.split('|')]
        positions = [p.strip() for p in positions_str.split('|')] if positions_str else [''] * len(players)
        dates = [d.strip() for d in dates_str.split('|')] if dates_str else [''] * len(players)
        ratings = [r.strip() for r in ratings_str.split('|')] if ratings_str else [''] * len(players)
        stars = [s.strip() for s in stars_str.split('|')] if stars_str else [''] * len(players)
        eligibility = [e.strip() for e in eligibility_str.split('|')] if eligibility_str else [''] * len(players)
        
        # Pad shorter lists to match players list length
        max_len = len(players)
        positions += [''] * (max_len - len(positions))
        dates += [''] * (max_len - len(dates))
        ratings += [''] * (max_len - len(ratings))
        stars += [''] * (max_len - len(stars))
        eligibility += [''] * (max_len - len(eligibility))
        
        # Group players by unique identity
        player_groups = defaultdict(list)
        
        for i, player in enumerate(players):
            players_processed += 1
            
            # Check if this player is already in our groups
            merged = False
            for existing_player in list(player_groups.keys()):
                if are_same_person(player, existing_player):
                    # Merge with existing entry
                    player_groups[existing_player].append({
                        'player': player,
                        'position': positions[i],
                        'date': dates[i],
                        'rating': ratings[i],
                        'stars': stars[i],
                        'eligibility': eligibility[i]
                    })
                    duplicates_merged += 1
                    merged = True
                    break
            
            if not merged:
                # Create new entry
                player_groups[player].append({
                    'player': player,
                    'position': positions[i],
                    'date': dates[i],
                    'rating': ratings[i],
                    'stars': stars[i],
                    'eligibility': eligibility[i]
                })
        
        # Create one edge per unique player
        # KEY FIX: Use MultiDiGraph or unique edge keys
        for player_name, player_data_list in player_groups.items():
            # For duplicate entries, take the first non-empty value for each attribute
            def get_best_value(attr_name):
                values = [pd[attr_name] for pd in player_data_list]
                for v in values:
                    if v and v != '' and v != 'None':
                        return v
                return ''
            
            final_data = {
                'players': player_name,
                'positions': get_best_value('position'),
                'dates': get_best_value('date'),
                'ratings': get_best_value('rating'),
                'stars': get_best_value('stars'),
                'eligibility': get_best_value('eligibility'),
                'weight': 1
            }
            
            # Add edge (MultiDiGraph automatically handles multiple edges)
            G_new.add_edge(source, target, **final_data)
            edges_after += 1
    
    print(f"  Edges: {edges_before} → {edges_after} (+{edges_after - edges_before})")
    print(f"  Players processed: {players_processed}")
    print(f"  Duplicates merged: {duplicates_merged}") 
    print()
    
    cleaned_graphs[name] = G_new

# ============================================
# STEP 4: IMPUTE MISSING RATINGS WITH STAR VALUES
# ============================================

print("\n" + "="*60)
print("IMPUTING MISSING RATINGS WITH STAR-TO-RATING VALUES")
print("="*60 + "\n")

for name, graph in cleaned_graphs.items():
    print(f"Processing {name}...")
    
    imputed_count = 0
    total_players = 0
    
    for s, t, data in graph.edges(data=True):
        rating_str = data.get('ratings', '')
        star_str = data.get('stars', '')
        
        total_players += 1
        
        # If rating is missing but star exists, use fallback
        if (not rating_str or rating_str == '' or rating_str == 'None') and star_str and star_str != '' and star_str != 'None':
            fallback = get_fallback_rating(star_str, method='median')
            data['ratings'] = str(fallback)
            imputed_count += 1
    
    print(f"  Imputed {imputed_count} out of {total_players} player ratings")

# ============================================
# STEP 5: SAVE CLEANED GRAPHS
# ============================================

print("\n" + "="*60)
print("SAVING CLEANED GRAPHS")
print("="*60 + "\n")

output_path = Path(r"C:\Users\User\Documents\cfb project\data\transferportal\cleaned")
output_path.mkdir(parents=True, exist_ok=True)

for name, graph in cleaned_graphs.items():
    output_file = output_path / f"{name}_cleaned.graphml"
    nx.write_graphml(graph, output_file)
    print(f"Saved: {output_file}")
    print(f"  Nodes: {len(graph.nodes())}")
    print(f"  Edges: {len(graph.edges())}")
    print()

print("✓ All graphs cleaned and saved!")

# ============================================
# STEP 6: SUMMARY STATISTICS
# ============================================

print("="*60)
print("FINAL SUMMARY")
print("="*60 + "\n")

summary_data = []
for name in xferfiles.keys():
    original = xferfiles[name]
    cleaned = cleaned_graphs[name]
    
    orig_edges = len(original.edges())
    clean_edges = len(cleaned.edges())
    
    summary_data.append({
        'Graph': name,
        'Original Edges': orig_edges,
        'Cleaned Edges': clean_edges,
        'Change': clean_edges - orig_edges,
        'Percent Increase': ((clean_edges - orig_edges) / orig_edges * 100) if orig_edges > 0 else 0
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# ============================================
# STEP 7: EXPORT MAPPING TABLE
# ============================================

print("\n" + "="*60)
print("STAR-TO-RATING MAPPING TABLE")
print("="*60 + "\n")

mapping_df = pd.DataFrame.from_dict(star_rating_map, orient='index')
mapping_df.index.name = 'Stars'
print(mapping_df.to_string())

# Save mapping to CSV
mapping_csv = output_path / "star_rating_mapping.csv"
mapping_df.to_csv(mapping_csv)
print(f"\n✓ Mapping table saved to: {mapping_csv}")

Loaded 5 transfer portal files

ANALYZING CURRENT DATA STATE

transfer_portal_2021:
  Edges: 958
  Players: 1053
  Multi-player edges: 77

transfer_portal_2022:
  Edges: 1225
  Players: 1367
  Multi-player edges: 113

transfer_portal_2023:
  Edges: 1413
  Players: 1607
  Multi-player edges: 140

transfer_portal_2024:
  Edges: 2340
  Players: 2654
  Multi-player edges: 224

transfer_portal_2025:
  Edges: 3296
  Players: 3765
  Multi-player edges: 337

BUILDING STAR-TO-RATING MAPPING FROM EXISTING DATA

Rating statistics by star level:
------------------------------------------------------------
1 Stars: No data available

2 Stars:
  Count: 108
  Mean:   0.7850
  Median: 0.7900
  Std:    0.0109
  Range:  0.7300 - 0.7900

3 Stars:
  Count: 5476
  Mean:   0.8496
  Median: 0.8500
  Std:    0.0236
  Range:  0.8000 - 0.8900

4 Stars:
  Count: 655
  Mean:   0.9125
  Median: 0.9100
  Std:    0.0146
  Range:  0.9000 - 0.9700

5 Stars:
  Count: 16
  Mean:   0.9850
  Median: 0.9800
  Std:    0.007

In [ ]:
# cleaning rec



In [6]:
# checking missing attributes

xferpathclean = Path(r"C:\Users\User\Documents\cfb project\data\transferportal\cleaned")
xferfilesclean = {}

for file_path in xferpathclean.glob('*.graphml'):
    name = file_path.stem
    G_x = nx.read_graphml(file_path)
    xferfilesclean[name] = G_x

# ============================================
# FUNCTION TO ANALYZE MISSING ATTRIBUTES
# ============================================

def analyze_missing_attributes(graph, graph_name):
    """
    Analyzes a graph for missing node and edge attributes
    Returns dictionaries with missing attribute information
    """
    print(f"{'='*60}")
    print(f"Analyzing: {graph_name}")
    print(f"{'='*60}\n")
    
    # --- NODE ATTRIBUTES ---
    print("NODE ATTRIBUTES:")
    print("-" * 40)
    
    # Collect all possible node attributes
    all_node_attrs = set()
    for node, data in graph.nodes(data=True):
        all_node_attrs.update(data.keys())
    
    print(f"Total unique node attributes found: {len(all_node_attrs)}")
    print(f"Attributes: {sorted(all_node_attrs)}\n")
    
    # Check which nodes are missing which attributes
    node_missing_summary = {}
    for attr in all_node_attrs:
        missing_nodes = []
        for node, data in graph.nodes(data=True):
            if attr not in data or data[attr] is None or data[attr] == '' or data[attr] == 0.0 or data[attr] == 'Unknown':
                missing_nodes.append(node)
        
        if missing_nodes:
            node_missing_summary[attr] = {
                'count': len(missing_nodes),
                'percentage': (len(missing_nodes) / len(graph.nodes())) * 100,
                'sample_nodes': missing_nodes[:5]  # First 5 examples
            }
    
    if node_missing_summary:
        print("Missing Node Attributes Summary:")
        for attr, info in sorted(node_missing_summary.items()):
            print(f"  {attr}:")
            print(f"    Missing in {info['count']}/{len(graph.nodes())} nodes ({info['percentage']:.1f}%)")
            print(f"    Sample nodes: {info['sample_nodes']}")
    else:
        print("✓ All nodes have all attributes!")
    
    print()
    
    # --- EDGE ATTRIBUTES ---
    print("EDGE ATTRIBUTES:")
    print("-" * 40)
    
    # Collect all possible edge attributes
    all_edge_attrs = set()
    for s, t, data in graph.edges(data=True):
        all_edge_attrs.update(data.keys())
    
    print(f"Total unique edge attributes found: {len(all_edge_attrs)}")
    print(f"Attributes: {sorted(all_edge_attrs)}\n")
    
    # Check which edges are missing which attributes
    edge_missing_summary = {}
    for attr in all_edge_attrs:
        missing_edges = []
        for s, t, data in graph.edges(data=True):
            if attr not in data or data[attr] is None or data[attr] == '' or data[attr] == 0.0 or data[attr] == 'Unknown':
                missing_edges.append((s, t))
        
        if missing_edges:
            edge_missing_summary[attr] = {
                'count': len(missing_edges),
                'percentage': (len(missing_edges) / len(graph.edges())) * 100,
                'sample_edges': missing_edges[:5]  # First 5 examples
            }
    
    if edge_missing_summary:
        print("Missing Edge Attributes Summary:")
        for attr, info in sorted(edge_missing_summary.items()):
            print(f"  {attr}:")
            print(f"    Missing in {info['count']}/{len(graph.edges())} edges ({info['percentage']:.1f}%)")
            print(f"    Sample edges: {info['sample_edges']}")
    else:
        print("✓ All edges have all attributes!")
    
    print("\n")
    
    return {
        'node_attrs': all_node_attrs,
        'edge_attrs': all_edge_attrs,
        'node_missing': node_missing_summary,
        'edge_missing': edge_missing_summary
    }


# ============================================
# ANALYZE ALL GRAPHS
# ============================================

all_results = {}

# Analyze transfer portal graphs
print("\n" + "="*60)
print("TRANSFER PORTAL GRAPHS")
print("="*60 + "\n")

for name, graph in xferfilesclean.items():
    results = analyze_missing_attributes(graph, name)
    all_results[name] = results

# Analyze recruiting graphs
#print("\n" + "="*60)
#print("RECRUITING GRAPHS")
#print("="*60 + "\n")

#for name, graph in recfiles.items():
 #   results = analyze_missing_attributes(graph, name)
  #  all_results[name] = results


# ============================================
# CREATE SUMMARY DATAFRAME
# ============================================

print("\n" + "="*60)
print("OVERALL SUMMARY")
print("="*60 + "\n")

summary_data = []
for graph_name, results in all_results.items():
    summary_data.append({
        'Graph': graph_name,
        'Node Attrs': len(results['node_attrs']),
        'Edge Attrs': len(results['edge_attrs']),
        'Missing Node Attrs': len(results['node_missing']),
        'Missing Edge Attrs': len(results['edge_missing'])
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))


TRANSFER PORTAL GRAPHS

Analyzing: transfer_portal_2021_cleaned

NODE ATTRIBUTES:
----------------------------------------
Total unique node attributes found: 4
Attributes: ['classification', 'conference', 'latitude', 'longitude']

Missing Node Attributes Summary:
  classification:
    Missing in 19/264 nodes (7.2%)
    Sample nodes: ['Missouri Western State', 'Winston-Salem State', 'City College of San Francisco', 'Mississippi Gulf Coast C.C.', 'Texas-Permian Basin']
  conference:
    Missing in 19/264 nodes (7.2%)
    Sample nodes: ['Missouri Western State', 'Winston-Salem State', 'City College of San Francisco', 'Mississippi Gulf Coast C.C.', 'Texas-Permian Basin']
  latitude:
    Missing in 23/264 nodes (8.7%)
    Sample nodes: ['Missouri Western State', 'Winston-Salem State', 'Northwestern', 'City College of San Francisco', 'Mississippi Gulf Coast C.C.']
  longitude:
    Missing in 23/264 nodes (8.7%)
    Sample nodes: ['Missouri Western State', 'Winston-Salem State', 'Northweste

In [7]:
import networkx as nx
import pandas as pd
import numpy as np
from pathlib import Path

# Load cleaned transfer portal data
xferpath = Path(r"C:\Users\User\Documents\cfb project\data\transferportal\cleaned")
xferfiles = {}

for file_path in xferpath.glob('*_cleaned.graphml'):
    name = file_path.stem.replace('_cleaned', '')
    G = nx.read_graphml(file_path)
    xferfiles[name] = G

print(f"Loaded {len(xferfiles)} transfer portal files\n")

# ============================================
# ANALYZE MISSING DATA PATTERNS
# ============================================

print("="*70)
print("MISSING DATA ANALYSIS")
print("="*70 + "\n")

missing_analysis = []

for name, G in xferfiles.items():
    year = int(name.split('_')[-1])
    
    total_edges = G.number_of_edges()
    missing_rating = 0
    missing_stars = 0
    has_rating = 0
    has_stars = 0
    
    # Track ratings by star level for missing data
    missing_by_star = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 'unknown': 0}
    present_by_star = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0}
    
    for u, v, data in G.edges(data=True):
        rating = data.get('ratings', '')
        stars = data.get('stars', '')
        
        # Check rating presence
        if not rating or rating in ['', '0.0', 'None']:
            missing_rating += 1
            
            # What star level is this missing rating?
            if stars and stars not in ['', 'None']:
                try:
                    star_val = int(float(stars))
                    if 1 <= star_val <= 5:
                        missing_by_star[star_val] += 1
                    else:
                        missing_by_star['unknown'] += 1
                except:
                    missing_by_star['unknown'] += 1
            else:
                missing_by_star['unknown'] += 1
        else:
            has_rating += 1
            
            # Track star distribution of present ratings
            if stars and stars not in ['', 'None']:
                try:
                    star_val = int(float(stars))
                    if 1 <= star_val <= 5:
                        present_by_star[star_val] += 1
                except:
                    pass
        
        # Check stars presence
        if not stars or stars in ['', 'None']:
            missing_stars += 1
        else:
            has_stars += 1
    
    missing_pct = (missing_rating / total_edges * 100) if total_edges > 0 else 0
    
    missing_analysis.append({
        'Year': year,
        'Total Edges': total_edges,
        'Missing Ratings': missing_rating,
        'Missing %': missing_pct,
        'Missing 2-star': missing_by_star[2],
        'Missing 3-star': missing_by_star[3],
        'Missing 4-star': missing_by_star[4],
        'Missing 5-star': missing_by_star[5],
        'Missing Unknown': missing_by_star['unknown'],
        'Present 2-star': present_by_star[2],
        'Present 3-star': present_by_star[3],
        'Present 4-star': present_by_star[4],
        'Present 5-star': present_by_star[5]
    })
    
    print(f"{year}:")
    print(f"  Total transfers: {total_edges}")
    print(f"  Missing ratings: {missing_rating} ({missing_pct:.1f}%)")
    print(f"  Missing by star level:")
    print(f"    2-star: {missing_by_star[2]}")
    print(f"    3-star: {missing_by_star[3]}")
    print(f"    4-star: {missing_by_star[4]}")
    print(f"    5-star: {missing_by_star[5]}")
    print(f"    Unknown: {missing_by_star['unknown']}")
    print()

missing_df = pd.DataFrame(missing_analysis)
print("\nSummary Table:")
print(missing_df.to_string(index=False))

# ============================================
# CALCULATE NPV WITH UNCERTAINTY BOUNDS
# ============================================

print("\n" + "="*70)
print("NPV CALCULATION WITH CONFIDENCE INTERVALS")
print("="*70 + "\n")

def calculate_npv_with_bounds(G, year):
    """
    Calculate NPV for each school with uncertainty bounds
    based on missing data.
    """
    school_stats = {}
    
    for node in G.nodes():
        in_transfers = []
        out_transfers = []
        in_missing = 0
        out_missing = 0
        
        # Incoming transfers
        for pred in G.predecessors(node):
            for key, data in G[pred][node].items() if isinstance(G, nx.MultiDiGraph) else [(0, G[pred][node])]:
                rating = data.get('ratings', '')
                if rating and rating not in ['', '0.0', 'None']:
                    in_transfers.append(float(rating))
                else:
                    in_missing += 1
        
        # Outgoing transfers
        for succ in G.successors(node):
            for key, data in G[node][succ].items() if isinstance(G, nx.MultiDiGraph) else [(0, G[node][succ])]:
                rating = data.get('ratings', '')
                if rating and rating not in ['', '0.0', 'None']:
                    out_transfers.append(float(rating))
                else:
                    out_missing += 1
        
        # Calculate NPV components
        in_sum = sum(in_transfers)
        out_sum = sum(out_transfers)
        npv = in_sum - out_sum
        
        # Calculate uncertainty bounds (assume missing values could be ±0.1)
        # Best case: missing incoming are high (0.95), missing outgoing are low (0.75)
        # Worst case: missing incoming are low (0.75), missing outgoing are high (0.95)
        best_case_npv = (in_sum + in_missing * 0.95) - (out_sum + out_missing * 0.75)
        worst_case_npv = (in_sum + in_missing * 0.75) - (out_sum + out_missing * 0.95)
        
        uncertainty = (best_case_npv - worst_case_npv) / 2
        
        school_stats[node] = {
            'npv': npv,
            'net_degree': len(in_transfers) - len(out_transfers),
            'in_count': len(in_transfers),
            'out_count': len(out_transfers),
            'in_missing': in_missing,
            'out_missing': out_missing,
            'total_missing': in_missing + out_missing,
            'uncertainty': uncertainty,
            'best_case': best_case_npv,
            'worst_case': worst_case_npv
        }
    
    return school_stats

# Calculate for most recent year
most_recent_year = max([int(name.split('_')[-1]) for name in xferfiles.keys()])
most_recent_graph_name = f"transfer_portal_{most_recent_year}"
G_recent = xferfiles[most_recent_graph_name]

stats = calculate_npv_with_bounds(G_recent, most_recent_year)

# Convert to DataFrame
results_df = pd.DataFrame.from_dict(stats, orient='index')
results_df['school'] = results_df.index
results_df = results_df.sort_values('npv', ascending=False)

print(f"Top 20 Schools by NPV ({most_recent_year}) with Uncertainty:")
print("="*70)
top20 = results_df.head(20)[['school', 'npv', 'net_degree', 'in_count', 'out_count', 
                               'total_missing', 'uncertainty', 'best_case', 'worst_case']]
print(top20.to_string(index=False))

print("\n" + "="*70)
print("KEY INSIGHTS:")
print("="*70)
print(f"\n1. Average uncertainty across all schools: ±{results_df['uncertainty'].mean():.2f}")
print(f"2. Max uncertainty: ±{results_df['uncertainty'].max():.2f} ({results_df.loc[results_df['uncertainty'].idxmax(), 'school']})")
print(f"3. Schools with >5 missing transfers: {len(results_df[results_df['total_missing'] > 5])}")

# Check ranking stability
print("\n4. Ranking Stability Analysis:")
results_df['rank'] = range(1, len(results_df) + 1)
results_df_best = results_df.copy()
results_df_best = results_df_best.sort_values('best_case', ascending=False)
results_df_best['rank_best'] = range(1, len(results_df_best) + 1)

results_df = results_df.merge(results_df_best[['school', 'rank_best']], on='school')
results_df['rank_change'] = abs(results_df['rank'] - results_df['rank_best'])

print(f"   Average rank change in best-case scenario: {results_df['rank_change'].mean():.1f} positions")
print(f"   Max rank change: {results_df['rank_change'].max():.0f} positions")
print(f"   Schools with rank change >5: {len(results_df[results_df['rank_change'] > 5])}")

print("\n5. Most Affected Schools (highest uncertainty):")
most_affected = results_df.nlargest(5, 'uncertainty')[['school', 'npv', 'total_missing', 'uncertainty', 'rank_change']]
print(most_affected.to_string(index=False))

Loaded 5 transfer portal files

MISSING DATA ANALYSIS

2021:
  Total transfers: 1051
  Missing ratings: 105 (10.0%)
  Missing by star level:
    2-star: 0
    3-star: 0
    4-star: 0
    5-star: 0
    Unknown: 104

2022:
  Total transfers: 1364
  Missing ratings: 123 (9.0%)
  Missing by star level:
    2-star: 0
    3-star: 0
    4-star: 0
    5-star: 0
    Unknown: 123

2023:
  Total transfers: 1603
  Missing ratings: 39 (2.4%)
  Missing by star level:
    2-star: 0
    3-star: 0
    4-star: 0
    5-star: 0
    Unknown: 39

2024:
  Total transfers: 2646
  Missing ratings: 213 (8.0%)
  Missing by star level:
    2-star: 0
    3-star: 0
    4-star: 0
    5-star: 0
    Unknown: 213

2025:
  Total transfers: 3751
  Missing ratings: 480 (12.8%)
  Missing by star level:
    2-star: 0
    3-star: 0
    4-star: 0
    5-star: 0
    Unknown: 480


Summary Table:
 Year  Total Edges  Missing Ratings  Missing %  Missing 2-star  Missing 3-star  Missing 4-star  Missing 5-star  Missing Unknown  Prese